In [ ]:
import torch
import numpy as np

# Tensori

In [ ]:
t1 = torch.tensor(4.)
t1

In [ ]:
t2 = torch.tensor([1., 2, 3, 4])
#NB! Tutti i dati di un tensore hanno sempre lo stesso tipo, in questo caso anche gli int diventano dei float

In [ ]:
t3 =torch.tensor([[5., 6],
                  [7, 8],
                  [9, 10]
                ])
t3

In [ ]:
t4 = torch.tensor([
    [[11, 12, 13],
     [13, 14, 15]],
    [[15, 16, 17],
     [17, 18, 19]]
])
t4

#crea una matrice tridimensionale 2x2x3 (2 "strati", 2 righe [righe in ogni strato], 3 colonne [elementi all'interno di ogni riga])

In [ ]:
print(t1)
t1.shape

In [ ]:
print(t2)
t2.shape

In [ ]:
print(t3)
t3.shape

In [ ]:
print(t4)
t4.shape

In [ ]:
# NB! NON si possono creare tensori con shape non definiti, ossia non si possono lasciare "spazi vuoti", se la prima riga
# è [1, 2, 3] la seconda non può essere ad esempio [5, 6] perché mancherebbe un elemento per completare la riga

In [ ]:
xt = torch.tensor (3. )
wt = torch. tensor (4., requires_grad =True)
bt = torch.tensor (5., requires_grad = True)
xt, wt, bt

# requires_grad =True specifica che siamo interessati a calcolare le derivate di questa variabile all'interno del programma
# se non lo mettiamo il programma non calcolerà mai la derivata rispetto quella determinata variabile

In [ ]:
yt = wt * xt + bt
yt

In [ ]:
yt.backward()

#esegue la derivata del tensore e salva i risultati all'interno della proprietà .grad del tensore

In [ ]:
print('dy/dx: ', xt.grad)
print('dy/dw: ', wt.grad)
print('dy/db: ', bt.grad)

In [ ]:
t6 = torch.full((3,2), 11)
t6

# torch.full((shape), num che inserisce)        serve a riempire un tensore con un numero specificando una shape

In [ ]:
t7 = torch.cat((t3, t6))
t7

# torch.cat((tensore1, tensore2))       permette di concatenare due tensori NB! devono avere shape compatibili

In [ ]:
t8 = torch.sin(t7)
t8

# torch.sin(tensore)     calcola il seno di ogni valore nel tensore

In [ ]:
t9 = t8.reshape(3, 2, 2)
t9

# tensore.reshape(3, 2, 2)      permette di assegnare una nuova shape a un tensore NB! devono essere compatibili il numero di dati

# Numpy

In [ ]:
a = np.array([[1,2],
              [3, 4]])
a

In [ ]:
b = torch.from_numpy(x)
b

# torch.from_numpy(var_numpy)       trasforma un vettore numpy in un tensore torch

In [ ]:
c = b.numpy()
c

# tensore.numpy()       trasforma un tensore in un array numpy

# Regressione lineare

In una regressione lineare ogni target è stimato da una somma pesata delle variabili in input a cui è sommata una distorsione detta bias
Il training della regressione lineare avviene tramite il settaggio dei pesi e dei bias affinchè la predizione di nuove informazioni sia accurata

In [ ]:
# Input (temp, precipitazioni, umidità)

inputs = np.array([[72, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]],
                   dtype = 'float32'
                  )

In [ ]:
# obiettivi (mele, arance)

obiettivi = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]],
                    dtype = 'float32'
                   )

Solitamente i dati vengono importati da file csv in array numpy che vengono processati e infine convertiti in PyTorch

In [ ]:
inputs = torch.from_numpy(inputs)
obiettivi = torch.from_numpy(obiettivi)

print(inputs)
print(obiettivi)

In [ ]:
pesi = torch.randn(2, 3, requires_grad = True)
bias = torch.randn(2, requires_grad = True)

# torch.randn(shape, requires_grad = True)      crea un tensore con la shape data con elementi random che hanno una distribuzione normalizzata

print(pesi)
print(bias)

Funzione che fa una predizione delle variabili target in base agli input dati

In [ ]:
def model(x):
    return x @ pesi.t() + bias

# @ moltiplicazione matriciale in PyTorch
# .t metodo che fa la trasposta del tensore

In [ ]:
pred = model(inputs)
print(pred)

Per migliorare la predizione dobbiamo prima compararla al target che ci si aspettava attraverso una funzione di costo come l'errore medio quadratico

In [ ]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

#torch.sum      restituisce la somma di tutti gli elementi del tensore
# tensore.numel     restituisce numero di elementi all'interno del tensore

In [ ]:
perdita = mse(pred, obiettivi)
print(perdita)

Si ha una predizione migliore quanto più è minore la perdita calcolata

In [ ]:
perdita.backward()

In [ ]:
print(pesi)
print(pesi.grad)

In [ ]:
print(bias)
print(bias.grad)

In [ ]:
with torch.no_grad():
    pesi -= pesi.grad * 1e-5
    bias -= bias.grad * 1e-5

# with torch.no_grad():         serve per non modificare la derivata mentre si aggiornano pesi e bias

In [ ]:
pred = model(inputs)
perdita = mse(pred, obiettivi)
print(perdita)

# la perdita diminuisce rispetto alla perdita precedente

In [ ]:
pesi.grad.zero_()
bias.grad.zero_()

print(pesi.grad)
print(bias.grad)

# bisogna resettare le derivate a zero perché pytorch accumula le derivate (derivata prima, seconda...)

## Training

1. gerenare delle predizioni
2. calcolare la perdita
3. calcolare le derivata dei pesi e dei bias
4. calibrare pesi e bias sottraendo una piccola porzione della derivata
5. resettare la derivata a zero

In [ ]:
# Generazione predizioni

pred = model(inputs)
print(pred)

In [ ]:
# Calcolo della perdita

perdita = mse(pred, obiettivi)
print(perdita)

In [ ]:
print(pesi)
print(bias)

In [ ]:
# Calcolo derivata

perdita.backward()
print(pesi.grad)
print(bias.grad)

In [ ]:
# Calibrazione pesi e bias

with torch.no_grad():
    pesi -= pesi.grad * 1e-5        #perché mi da errore: variable is 'unbound' ma il programma funziona lo stesso
    bias -= bias.grad * 1e-5

In [ ]:
# Azzeramento pesi e bias

pesi.grad.zero_()
bias.grad.zero_()

In [ ]:
# visualizzazione nuovi pesi e bias calibrati

print(pesi)
print(bias)

In [ ]:
pred = model(inputs)
perdita = mse(pred, obiettivi)
print(perdita)

## Training per più epochs

In [ ]:
for i in range(100):
    pred = model(inputs)
    perdita = mse(pred, obiettivi)
    perdita.backward()

    with torch.no_grad():
        pesi -= pesi.grad * 1e-5
        bias -= bias.grad * 1e-5
        pesi.grad.zero_()
        bias.grad.zero_()

In [ ]:
pred = model(inputs)
perdita = mse(pred, obiettivi)
print(perdita)

In [ ]:
pred

In [ ]:
obiettivi

# Regressione lineare usando funzioni già presenti su PyTorch

In [1375]:
import torch.nn as nn

In [1402]:
# inizializzazione input e obiettivi


inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 86, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                   dtype = 'float32'
                  )

obiettivi = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                    dtype = 'float32'
                   )

In [1403]:
# Trasofrmazione degli input e obiettivi a tensori
inputs = torch.from_numpy(inputs)
obiettivi = torch.from_numpy(obiettivi)


## Dataset, DataLoader e regressione lineare

In [1404]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [1405]:
# definizione del dataset
train_ds = TensorDataset(inputs, obiettivi)
train_ds[0:3]

#TensorDataSet permette di accedere a piccole porzioni del data training usando la notazione array

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [1406]:
# definizione data loader

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

# DataLoader spezza il set di dati in lotti di una dimensione predefinita durante il training

In [1407]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [103.,  43.,  36.],
        [ 74.,  66.,  43.],
        [ 69.,  96.,  70.],
        [ 88., 134.,  59.]])
tensor([[ 22.,  37.],
        [ 20.,  38.],
        [ 57.,  69.],
        [103., 119.],
        [118., 132.]])


In [1408]:
# definizione del modello

model = nn.Linear(3, 2)     #nn.Linear(numero input, numero output)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3171, -0.3306, -0.1153],
        [-0.5098,  0.1090, -0.1359]], requires_grad=True)
Parameter containing:
tensor([-0.3125,  0.5537], requires_grad=True)


In [1409]:
pred = model(inputs)
print(pred)

tensor([[ -4.2754, -35.2066],
        [ -7.9321, -44.9494],
        [-23.7165, -37.0815],
        [ 13.5454, -51.7909],
        [-18.2438, -33.6774],
        [ -3.6277, -35.8253],
        [ -7.3862, -45.3033],
        [-23.5147, -37.7272],
        [ 12.8978, -51.1721],
        [-18.6762, -33.3035],
        [ -4.0601, -35.4515],
        [ -7.2845, -45.5682],
        [-23.9318, -36.8366],
        [ 13.9778, -52.1648],
        [-18.8915, -33.0586]], grad_fn=<AddmmBackward0>)


## Funzione di perdita

In [1410]:
import torch.nn.functional as F

In [1411]:
funz_perdita = F.mse_loss

# definizione funzione di perdita con mse

In [1412]:
perdita = funz_perdita(model(inputs), obiettivi)
print(perdita)

tensor(13910.0918, grad_fn=<MseLossBackward0>)


## Ottimizzazione perdita

In [1413]:
ott = torch.optim.SGD(model.parameters(), lr = 1e-5)

# Funzione per training

In [1414]:
def fit(num_epochs, model, funz_perdita, ott, train_dl):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            # 1. genreazione predizioni
            pred = model(xb)
            # 2. calcolo perdita
            perdita = funz_perdita(pred, yb)
            # 3. calcolo derivata
            perdita.backward()
            # 4. aggiornamento parametri usando derivate
            ott.step()
            # reset derivate a zero
            ott.zero_grad()
        if(epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Perdita: {:.4f}'.format(epoch+1, num_epochs, perdita.item()))

In [1415]:
fit(100, model, funz_perdita, ott, train_dl)

Epoch [10/100], Perdita: 764.8069
Epoch [20/100], Perdita: 338.8612
Epoch [30/100], Perdita: 378.7642
Epoch [40/100], Perdita: 170.5118
Epoch [50/100], Perdita: 119.1818
Epoch [60/100], Perdita: 88.0113
Epoch [70/100], Perdita: 51.4697
Epoch [80/100], Perdita: 46.7861
Epoch [90/100], Perdita: 60.8661
Epoch [100/100], Perdita: 22.1664


In [1416]:
pred = model(inputs)
pred

tensor([[ 58.9055,  71.3608],
        [ 81.1291,  97.7246],
        [118.2683, 138.2172],
        [ 31.3359,  42.0183],
        [ 94.0051, 111.1863],
        [ 57.8867,  70.2838],
        [ 79.8197,  96.3281],
        [118.4608, 138.5244],
        [ 32.3546,  43.0953],
        [ 94.5546, 111.7937],
        [ 58.4363,  70.8912],
        [ 80.1103,  96.6476],
        [118.7375, 138.6868],
        [ 30.7863,  41.4109],
        [ 95.0238, 112.2633]], grad_fn=<AddmmBackward0>)

In [1417]:
obiettivi

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])